<a href="https://colab.research.google.com/github/amirnafisa/decon4lindt/blob/master/Yolo_v1_1_0.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Yolo v3 Coco Dataset

## Setup

### Mount the google drive

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


### Get the Yolo v3 Model

In [2]:
!git clone https://github.com/ultralytics/yolov3

Cloning into 'yolov3'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 5950 (delta 0), reused 1 (delta 0), pack-reused 5947
Receiving objects: 100% (5950/5950), 6.16 MiB | 4.84 MiB/s, done.
Resolving deltas: 100% (4101/4101), done.


### Get the dataset

In [3]:
#https://drive.google.com/file/d/1XcPzYVLkipsuwOXFa1Qf1AZjVOT3aIL6/view?usp=sharing
!wget --load-cookies /tmp/cookies.txt \
"https://docs.google.com/uc?export=download&confirm=$(wget --quiet \
--save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate \
'https://docs.google.com/uc?export=download&id=1XcPzYVLkipsuwOXFa1Qf1AZjVOT3aIL6' \
-O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1XcPzYVLkipsuwOXFa1Qf1AZjVOT3aIL6"\
 -O coco.tar.gz && rm -rf /tmp/cookies.txt
!tar -zxvf coco.tar.gz

--2019-11-11 22:36:42--  https://docs.google.com/uc?export=download&confirm=RCpD&id=1XcPzYVLkipsuwOXFa1Qf1AZjVOT3aIL6
Resolving docs.google.com (docs.google.com)... 64.233.189.138, 64.233.189.101, 64.233.189.102, ...
Connecting to docs.google.com (docs.google.com)|64.233.189.138|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-0s-9g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/o721a0jqlnmiuhue3o2nen8v62ijveep/1573509600000/05677703835476312541/*/1XcPzYVLkipsuwOXFa1Qf1AZjVOT3aIL6?e=download [following]
--2019-11-11 22:36:42--  https://doc-0s-9g-docs.googleusercontent.com/docs/securesc/ha0ro937gcuc7l7deffksulhg5h7mbp1/o721a0jqlnmiuhue3o2nen8v62ijveep/1573509600000/05677703835476312541/*/1XcPzYVLkipsuwOXFa1Qf1AZjVOT3aIL6?e=download
Resolving doc-0s-9g-docs.googleusercontent.com (doc-0s-9g-docs.googleusercontent.com)... 108.177.125.132, 2404:6800:4008:c01::84
Connecting to doc-0s-9g-docs.googleusercontent.co

### Import dependencies

In [0]:
import requests
import csv
import urllib.request
import codecs
import os

import argparse

import torch.distributed as dist
import torch.optim as optim
import torch.optim.lr_scheduler as lr_scheduler

import json

from torch.utils.data import DataLoader

In [7]:
%cd yolov3/

/content/yolov3


In [4]:
#https://drive.google.com/file/d/1ofQ1xXgbvXqTGkykZv9-8V2HfSLj7ehp/view?usp=sharing
!wget --no-check-certificate \
'https://docs.google.com/uc?export=download&id=1ofQ1xXgbvXqTGkykZv9-8V2HfSLj7ehp' \
-O /content/yolov3/cfg/yolov3-custom.cfg
#!cp /content/drive/My\ Drive/yolov3-custom.cfg /content/yolov3/cfg/
# Modify yolov3-custom.cfg as follows: Here n = 2
'''Each YOLO layer has 255 outputs: 85 outputs per anchor [4 box coordinates + 1 
object confidence + 80 class confidences], times 3 anchors. If you use fewer 
classes, you can reduce this to [4 + 1 + n] * 3 = 15 + 3*n outputs, where n is 
your class count. This modification should be made to the output filter 
preceding each of the 3 YOLO layers. Also modify classes=80 to classes=n in 
each YOLO layer, where n is your class count.'''


'Each YOLO layer has 255 outputs: 85 outputs per anchor [4 box coordinates + 1 \nobject confidence + 80 class confidences], times 3 anchors. If you use fewer \nclasses, you can reduce this to [4 + 1 + n] * 3 = 15 + 3*n outputs, where n is \nyour class count. This modification should be made to the output filter \npreceding each of the 3 YOLO layers. Also modify classes=80 to classes=n in \neach YOLO layer, where n is your class count.'

In [0]:
!touch data/coco.names
!echo "Door handle" >> data/coco.names
!echo "Light switch" >> data/coco.names

In [9]:
#Download pretrained weights
!bash weights/download_yolov3_weights.sh

--2019-11-11 23:01:25--  https://pjreddie.com/media/files/yolov3.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2019-11-11 23:01:26--  https://pjreddie.com/media/files/yolov3-tiny.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retrieved; nothing to do.

--2019-11-11 23:01:26--  https://pjreddie.com/media/files/yolov3-spp.weights
Resolving pjreddie.com (pjreddie.com)... 128.208.4.108
Connecting to pjreddie.com (pjreddie.com)|128.208.4.108|:443... connected.
HTTP request sent, awaiting response... 416 Requested Range Not Satisfiable

    The file is already fully retriev

## Sample run with 10 image dataset

### Prepare small dataset paths

In [0]:
!touch data/coco_10img.data
!echo "classes=2" >> data/coco_10img.data
!echo "train=../coco/coco_10img.txt" >> data/coco_10img.data
!echo "valid=../coco/coco_10img.txt" >> data/coco_10img.data
!echo "names=data/coco.names" >> data/coco_10img.data
!echo "backup=backup/" >> data/coco_10img.data
!echo "eval=coco" >> data/coco_10img.data

In [0]:
!touch ../coco/coco_10img.txt
!echo "../coco/images/train2014/002138ea46a43c8e.jpg" >> ../coco/coco_10img.txt
!echo "../coco/images/train2014/00334787ad24bb0d.jpg" >> ../coco/coco_10img.txt
!echo "../coco/images/train2014/003a2b5030034d00.jpg" >> ../coco/coco_10img.txt
!echo "../coco/images/train2014/003a823d6df254e2.jpg" >> ../coco/coco_10img.txt
!echo "../coco/images/train2014/003e7378c5d68691.jpg" >> ../coco/coco_10img.txt
!echo "../coco/images/train2014/0058083847c4e459.jpg" >> ../coco/coco_10img.txt
!echo "../coco/images/train2014/00a705bdfc1df5a3.jpg" >> ../coco/coco_10img.txt
!echo "../coco/images/train2014/013e136304a230a7.jpg" >> ../coco/coco_10img.txt
!echo "../coco/images/train2014/043541a95a0d1cfa.jpg" >> ../coco/coco_10img.txt
!echo "../coco/images/train2014/05f5fdc84c2d31ef.jpg" >> ../coco/coco_10img.txt

### Train

In [9]:
!python3 train.py --data data/coco_10img.data --cfg cfg/yolov3-custom.cfg --weights \
weights/yolov3-spp.weights --transfer --epochs 10 --batch-size 4

Namespace(accumulate=2, adam=False, arc='default', batch_size=4, bucket='', cache_images=False, cfg='cfg/yolov3-custom.cfg', data='data/coco_10img.data', device='', epochs=10, evolve=False, img_size=416, img_weights=False, multi_scale=False, name='', nosave=False, notest=False, prebias=False, rect=False, resume=False, transfer=True, var=None, weights='weights/yolov3-spp.weights')
Using CUDA device0 _CudaDeviceProperties(name='Tesla K80', total_memory=11441MB)

Traceback (most recent call last):
  File "train.py", line 436, in <module>
    train()  # train normally
  File "train.py", line 74, in train
    data_dict = parse_data_cfg(data)
  File "/content/yolov3/utils/parse_config.py", line 32, in parse_data_cfg
    with open(path, 'r') as fp:
FileNotFoundError: [Errno 2] No such file or directory: 'data/coco_10img.data'


### Detect a sample image

In [18]:
!cp ../coco/images/train2014/eb6b084e7bdb9e23.jpg data/samples/test.jpg
!python3 detect.py --data data/custom_full.data --cfg cfg/yolov3-custom.cfg --output results --weights weights/last.pt

Namespace(cfg='cfg/yolov3-custom.cfg', conf_thres=0.3, data='data/coco_10img.data', device='', fourcc='mp4v', half=False, img_size=416, nms_thres=0.5, output='results', source='data/samples', view_img=False, weights='weights/last.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

image 1/3 data/samples/bus.jpg: 416x320 Done. (0.026s)
image 2/3 data/samples/test.jpg: 416x416 Done. (0.020s)
image 3/3 data/samples/zidane.jpg: 256x416 Done. (0.017s)
Results saved to /content/yolov3/results
Done. (0.235s)


## Run on full custom dataset

### Prepare dataset path

In [0]:
!touch data/custom_full.data
!echo "classes=2" >> data/custom_full.data
!echo "train=../coco/coco_full_train.txt" >> data/custom_full.data
!echo "valid=../coco/coco_full_val.txt" >> data/custom_full.data
!echo "names=data/coco.names" >> data/custom_full.data
!echo "backup=backup/" >> data/custom_full.data
!echo "eval=coco" >> data/custom_full.data

### Train

In [11]:
!python3 train.py --data data/custom_full.data --cfg cfg/yolov3-custom.cfg --weights \
weights/yolov3-spp.weights --transfer --epochs 10 --batch-size 64

Namespace(accumulate=2, adam=False, arc='default', batch_size=64, bucket='', cache_images=False, cfg='cfg/yolov3-custom.cfg', data='data/custom_full.data', device='', epochs=10, evolve=False, img_size=416, img_weights=False, multi_scale=False, name='', nosave=False, notest=False, prebias=False, rect=False, resume=False, transfer=True, var=None, weights='weights/yolov3-spp.weights')
Using CUDA device0 _CudaDeviceProperties(name='Tesla P100-PCIE-16GB', total_memory=16280MB)

Model Summary: 225 layers, 6.25787e+07 parameters, 37695 gradients
Starting training for 10 epochs...

     Epoch   gpu_mem      GIoU       obj       cls     total   targets  img_size
       0/9     5.58G      7.23        59      83.1       149        55       416: 100% 10/10 [02:35<00:00, 12.58s/it]
Reading image shapes: 100% 36/36 [00:00<00:00, 682.90it/s]
               Class    Images   Targets         P         R   mAP@0.5        F1: 100% 1/1 [00:05<00:00,  5.66s/it]
                 all        36        36     

### Detect a sample image

In [18]:
!cp ../coco/images/train2014/eb6b084e7bdb9e23.jpg data/samples/test.jpg
!python3 detect.py --data data/custom_full.data --cfg cfg/yolov3-custom.cfg --output results --weights weights/last.pt

Namespace(cfg='cfg/yolov3-custom.cfg', conf_thres=0.3, data='data/custom_full.data', device='', fourcc='mp4v', half=False, img_size=416, nms_thres=0.5, output='results', source='data/samples', view_img=False, weights='weights/last.pt')
Using CUDA device0 _CudaDeviceProperties(name='Tesla K80', total_memory=11441MB)

image 1/3 data/samples/bus.jpg: 416x320 Done. (0.088s)
image 2/3 data/samples/test.jpg: 416x416 Done. (0.091s)
image 3/3 data/samples/zidane.jpg: 256x416 Done. (0.065s)
Results saved to /content/yolov3/results
Done. (0.464s)


# Sanity Check

In [0]:
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import torch
import numpy as np

imgs=np.array([mpimg.imread('data/samples/test.jpg')])
targets = torch.tensor([[0, 0, 0.45906250000000004, 0.54375, 0.36062500000000003, 0.32500000000000007]])

def xywh2xyxy(x):
    # Convert bounding box format from [x, y, w, h] to [x1, y1, x2, y2]
    y = torch.zeros_like(x) if isinstance(x, torch.Tensor) else np.zeros_like(x)
    y[:, 0] = x[:, 0] - x[:, 2] / 2
    y[:, 1] = x[:, 1] - x[:, 3] / 2
    y[:, 2] = x[:, 0] + x[:, 2] / 2
    y[:, 3] = x[:, 1] + x[:, 3] / 2
    return y


boxes = xywh2xyxy(targets[targets[:, 0] == 0, 2:6]).T
bs, w, h, _ = imgs.shape  # batch size, _, height, width
bs = min(bs, 16)  # limit plot to 16 images
ns = np.ceil(bs ** 0.5)  # number of subplots

fig = plt.figure(figsize=(10, 10))
for i in range(bs):
    boxes = xywh2xyxy(targets[targets[:, 0] == i, 2:6]).T
    boxes[[0, 2]] *= w
    boxes[[1, 3]] *= h
    plt.subplot(ns, ns, i + 1).imshow(imgs[i])
    plt.plot(boxes[[0, 2, 2, 0, 0]], boxes[[1, 1, 3, 3, 1]], '.-')
    plt.axis('off')
        
fig.tight_layout()
plt.show()